In [1]:
%matplotlib inline
%pylab
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%config InlineBackend.figure_format='retina'

import numpy as np
import os
from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib.colors import LogNorm

Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


## Harmonic mean

In [2]:
import statistics

In [3]:
print(statistics.harmonic_mean([1.35e3, 0.997e3])) # lyso, water

1146.9535577332765


**Absorbance**: from lys in gly sol

In [4]:
T_cap = abs(2780-2400) / 2780
T_cap

0.1366906474820144

In [5]:
T_test = abs(2780-1500) / 2780
# print(T_test)
# A_test = abs(2400-1500) / 2400
# A = 2 - np.log10(T_test*100)
A = 2 - np.log10(0.495*100)
# A = 2 - np.log10(0.673*100)
A

0.30539480106643135

# Get dscan (100 um2 beam size)

In [6]:
!ls ../02-sources

elog			HydLys_0p25_1_00001.fio  img_1_.npy
fit_params_h25_LLT.npz	img_1000.npy		 img_avg_blank.npy
fit_params_h25_old.npz	img_100_blank.npy	 img_avg.npy
fit_pars_Temps.pkl	img_100.npy		 waxs_temps.npz


In [7]:
#!head -n 164 ../02-sources/HydLys_0p25_1_00001.fio  | tail -93

In [8]:
#!cat -n ../02-sources/HydLys_0p25_1_00001.fio 

### Protein volume fraction

In [9]:
h = 0.28 # hydration level
rhow = 0.997e3 # water density
rhop = 1.35e3 # lysozyme density

# volume fraction
Vf = (rhow/rhop/h) / (1+ rhow/rhop/h)
Vf

0.7250909090909091

## Compute dose

In [34]:
flux = 6e9 # photons/sec
# A = 0.85 # absorption
# A = 0.54 # absorption what mario used
A = 0.31 # from 2-log(transm=0.495)
# Ec = 12.4e3 * 1.602e-19  # J Photon energy 
Ec = 9e3 * 1.602e-19  # J Photon energy 
rho_p = 1.35e3 # kg / m3 sample mass density
rho = statistics.harmonic_mean([1.35e3, 0.997e3])
phi = 0.73
rho_w = np.average([1.35e3, 0.997e3], weights=[phi, 1-phi])
print("density: ", rho_w)
# phi = 0.678 # volume fraction

te  = 1 # s exposure time
z = 30e-6 # m
ds = 1.5e-3 # m sample thickness

dose_christian = flux * Ec * A / (z**2 * ds * rho_w) * te # this
# dose_christian2 = flux * Ec * A2 / (z**2 * ds * rho) * te
dose_phi = flux * Ec * A * phi / (z**2 * ds * rho_p) * te

print("no phi: %.2f" %dose_christian) # this used !!
print("yes phi: %.2f" %dose_phi)
# print("no phi 2: %.2f" %dose_christian2)

density:  1254.69
no phi: 1583.24
yes phi: 1074.17


In [35]:
flux = 6e9 # photons/sec
A = 0.85 # absorption
Ec = 12.4e3 * 1.602e-19  # J Photon energy 
rho = 1.35e3 # kg / m3 sample mass density 
phi = 0.678 #0.8 # volume fraction
te  = 1 # s exposure time
z = 30e-6 # m
ds = 1.5e-3 # m sample thickness

dose = flux * Ec * A * phi / (z**2 * ds * rho) * te
print("%.2f" %dose)

3768.92


# flux and fluences

**Beamtime 1 (2021)**

In [37]:
25*16

400

In [45]:
flux1 = 4e9 
beam1 = 30*30
Tr1 = [1, 0.75, 0.55, 0.20, 0.09] # 0,3,6,16,24

In [46]:
print("Flux (ph/s) \tFluence(ph/s/um2) \tTransm")
for tr in Tr1:
    fl = flux1 * tr / beam1
    print(f"{flux1*tr:.1e} \t{fl:.1e} \t\t{tr:.2f}")

Flux (ph/s) 	Fluence(ph/s/um2) 	Transm
4.0e+09 	4.4e+06 		1.00
3.0e+09 	3.3e+06 		0.75
2.2e+09 	2.4e+06 		0.55
8.0e+08 	8.9e+05 		0.20
3.6e+08 	4.0e+05 		0.09


**Beamtime 2 (2022)**

0,3,6,9, used 6 for temps

In [47]:
flux2 = 6e9
beam2 = 30*30
Tr2 = [1, 0.45, 0.22, 0.1] # 0,3,6,9

In [48]:
print("Flux (ph/s) \tFluence(ph/s/um2) \tTransm")
for tr in Tr2:
    fl = flux2 * tr / beam2
    print(f"{flux2*tr:.1e} \t{fl:.1e} \t\t{tr:.2f}")

Flux (ph/s) 	Fluence(ph/s/um2) 	Transm
6.0e+09 	6.7e+06 		1.00
2.7e+09 	3.0e+06 		0.45
1.3e+09 	1.5e+06 		0.22
6.0e+08 	6.7e+05 		0.10


# boh

In [12]:
time = np.load('../05-source/a_xtime.npy')
time2 = np.load('../05-source/a_xtime_long.npy')
#time = np.arange(0,1000,0.5)
doses = dose * time
doses2 = dose * time2

FileNotFoundError: [Errno 2] No such file or directory: '../05-source/a_xtime.npy'

In [ ]:
time2[-5:]

In [ ]:
plt.figure(figsize=(5,4))
tr = [1, 0.74, 0.55, 0.2, 0.09]
col = plt.cm.hot(np.linspace(0,1,len(tr)+2))
+1
for i in range(len(tr)-1):
    plt.plot(time, doses*1e-3*tr[i], c=col[i], label=f'{tr[i]}')
d = doses2*1e-3*tr[-1]
plt.plot(time2, doses2*1e-3*tr[-1], c=col[i+1], label=f'{tr[-1]}')
plt.axhline(y=d[-1], c='gray', lw=1, ls='--')

plt.legend(frameon=False, title='Attenuation')
plt.title(f'Dose rate {dose*1e-3:.1f} kGy/s')
plt.xlabel('time (s)', fontsize=16)
plt.ylabel('Dose (kGy)', fontsize=16)
#plt.yscale('log')
#plt.xscale('log')
plt.tight_layout()
#plt.savefig('../06-paper-plots/dose-vs-time.png', dpi=300)
plt.show()

In [ ]:
np.shape(doses)

In [ ]:
plt.figure(figsize=(5,4))
tr = [1, 0.74, 0.55, 0.2, 0.09]
#ttime = [1e3, 1e3, 1e3, 1e3, 2e3]
ttime = [1e3, 1e3, 1e3, 1e3, 1e3]
col = plt.cm.viridis(np.linspace(0,1,len(tr)))

for i in range(len(tr)):
    x = tr[i]
    y = dose*1e-6*tr[i]*ttime[i]
    plt.plot(x, y, c='k', marker='o',)
    print(f'{x}\t{y:.2f} MGy')

plt.plot(tr[-1], dose*1e-6*tr[i]*2e3, marker='x',c='k')
print(f'{tr[-1]}\t{dose*1e-6*tr[i]*2e3:.2f} MGy')

plt.xlabel('Transmission', fontsize=16)
plt.ylabel('Total absorbed dose (MGy)', fontsize=16)
plt.tight_layout()
#plt.savefig('../06-paper-plots/dose-vs-transm.png', dpi=300)
plt.show()